# 객담도말 결핵진단 딥러닝 모델기반 분류기

훈련된 모델을 사용하여 입력되는 이미지에 대해 분류합니다.

In [7]:
# import

import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.models import model_from_json
from keras.utils import np_utils

In [8]:
# configuraiton
MODEL_NAME = 'tb_cnn'
# MODEL_NAME = 'tb_vgglike'

INPUT_PATCH_IMAGE_DIR = './input/'

MODEL_ARCHITECUTRE_FILE_PATH = './save/' + MODEL_NAME + '_model_architecture.json'
MODEL_WEIGHTS_FILE_PATH = './save/' + MODEL_NAME + '_model_weights.h5'

FALSE_DECISION_THRESHOLD = 3

IMG_CHANNELS = 1
IMG_ROWS = 64
IMG_COLS = 64

NUM_CLASSES = 2

In [9]:
def GetPatchImageFileList(dir_path):

    patch_image_file_list = []
    
    for (path, dir, files) in os.walk(dir_path):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            if ext == '.bmp' or ext == '.png' or ext == '.jpg' or ext == '.jpeg':
                patch_image_file_list.append(dir_path + filename)
                    
    return patch_image_file_list

In [10]:
def GenerateDataset(patch_image_file_list, channel, row, col):
    
    buf = []
    
    for it in patch_image_file_list:
        cv_img = cv2.imread(it)
        cv_gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
        buf.append(cv_gray)
    
    dataset = np.array(buf)
    dataset = dataset.reshape(len(patch_image_file_list), channel, row, col)
    dataset = dataset.astype('float32')
    dataset /= 255
    
    return dataset

In [11]:
# architecture와 weight가 저장된 파일로부터 모델을 로딩합니다.

def LoadModel(model_architecture_file_path, model_weights_file_path):
    
    fp_json = open(model_architecture_file_path, 'r')
    json_string = fp_json.read()
    fp_json.close()
        
    model = model_from_json(json_string)
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

    model.load_weights(model_weights_file_path)
    # model.summary()
    
    return model

In [12]:
# 지정된 패치이미지 폴더에서 패치이미지를 로딩해서 데이터셋을 생성합니다.

patch_image_list = GetPatchImageFileList(INPUT_PATCH_IMAGE_DIR)
test_dataset = GenerateDataset(patch_image_list, IMG_CHANNELS, IMG_ROWS, IMG_COLS)

# 모델을 로딩합니다.
    
model = LoadModel(MODEL_ARCHITECUTRE_FILE_PATH, 
                   MODEL_WEIGHTS_FILE_PATH)

# 모델을 사용하여 분류합니다.
    
classes = model.predict_classes(test_dataset, batch_size=32)
classes_hist = np.histogram(classes, bins=range(NUM_CLASSES+1))

print('false object count : ' + str(classes_hist[0][0]))
print('true object count : ' + str(classes_hist[0][1]))

print('[Result]')

if classes_hist[0][1] > FALSE_DECISION_THRESHOLD :
    print('positive')
else:
    print('negative')

33/33 [==============================] - 0s     
false object count : 21
true object count : 12
[Result]
positive
